In [6]:
#Se importan todos los paquetes que se van a usar en código
import requests
import json
from IPython.display import JSON
from keys import *

In [7]:
#be careful not to run too often as there is a maximum of 200 api calls per month

# se obtienen la api para poder solicitar la información requerida
url = "https://aerodatabox.p.rapidapi.com/flights/airports/icao/CYUL/2025-09-01T08:00/2025-09-01T20:00"

querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"true","withLocation":"false"}

headers = {
    'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
    'x-rapidapi-key': flights_key
    }

response = requests.request("GET", url, headers=headers, params=querystring)

In [8]:
# Si el acceso es permitido se guarda en un archivo json para poder editar y agregarle información al archivo
if response.status_code == 200:
    new_data = response.json()

    try:
        with open("data.json", "r") as json_file:
            existing_data = json.load(json_file)
    except (FileNotFoundError, json.decoder.JSONDecodeError):
        existing_data = []

    existing_data.append(new_data)

    with open("data.json", "w") as json_file:
        json.dump(existing_data, json_file, indent=4)
        print("Data appended to data.json file.")
else:
    print("Failed to retrieve data from the API. Status code:", response.status_code)

Data appended to data.json file.


In [10]:
# Se crea la lista con los datos que se van a solicitar de la página web
flight_arrivals = response.json()

for flight in flight_arrivals:
    flight_arrivals['arrivals'][0]['arrival']['scheduledTime']
    flight_arrivals['arrivals'][0]['number']
    flight_arrivals['arrivals'][0]['arrival']
    flight_arrivals['arrivals'][0]['departure']['airport']['name']
    flight_arrivals['arrivals'][0]['aircraft']['model']
    flight_arrivals['arrivals'][0]['airline']['name']

In [11]:
# Se solicitan los datos dentro de cada apartado de la lista creada, despues se imprimen los primeros elementos de la lista 
import pandas as pd
from numpy import nan

flight_arrivals = response.json()

def flight_extraction(flights):
    flights_df = pd.json_normalize({
        'Scheduled_arrival_time': flights['arrival']['scheduledTime'],
        'flight_number': flights['number'],
        'from' : flights['departure']['airport']['name'],
        'airline' : flights['airline']['name'],
        'aircraft' : flights['aircraft']['model']
    })
    return(flights_df)

flight_arrivals = pd.concat(
    [flight_extraction(flight) for flight in flight_arrivals['arrivals']]
)

flight_arrivals.head()

,flight_number,from,airline,aircraft,Scheduled_arrival_time.utc,Scheduled_arrival_time.local
0,EK 1243,Dubai,Emirates,Boeing 777-300,2025-09-01 12:00Z,2025-09-01 08:00-04:00
0,EK 243,Dubai,Emirates,Boeing 777-200LR,2025-09-01 12:00Z,2025-09-01 08:00-04:00
0,AC 314,Vancouver,Air Canada,Boeing 737,2025-09-01 10:59Z,2025-09-01 06:59-04:00
0,PD 2453,Toronto,Porter,Bombardier Dash 8 Q400 / DHC-8-400,2025-09-01 12:15Z,2025-09-01 08:15-04:00
0,AC 400,Toronto,Air Canada,Airbus A220-300,2025-09-01 12:21Z,2025-09-01 08:21-04:00
